In [1]:
!pip -q install langchain huggingface_hub tiktoken
!pip -q install --upgrade together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.2 MB/s eta 0:00:00


## LLaMA 2 on Together API

In [114]:
import os

os.environ["TOGETHER_API_KEY"] = "api_key"

In [3]:
!pip show langchain

Name: langchain
Version: 0.0.338
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


# Setting up Together API


In [ ]:
import together

# set your API key
together.api_key = os.environ["TOGETHER_API_KEY"]

# list available models and descriptons
models = together.Models.list()

In [5]:
# print the first model's name
print(models[3]['name']), print(models[52]['name'])

EleutherAI/pythia-1b-v0
togethercomputer/GPT-JT-Moderation-6B


(None, None)

In [ ]:
together.Models.start("togethercomputer/llama-2-70b-chat")

In [9]:
import together

import logging
from typing import Any, Dict, List, Mapping, Optional

from pydantic import Extra, Field, root_validator

from langchain.callbacks.manager import CallbackManagerForLLMRun
from langchain.llms.base import LLM
from langchain.llms.utils import enforce_stop_tokens
from langchain.utils import get_from_dict_or_env

class TogetherLLM(LLM):
    """Together large language models."""

    model: str = "togethercomputer/llama-2-70b-chat"
    """model endpoint to use"""

    together_api_key: str = os.environ["TOGETHER_API_KEY"]
    """Together API key"""

    temperature: float = 0.7
    """What sampling temperature to use."""

    max_tokens: int = 512
    """The maximum number of tokens to generate in the completion."""

    class Config:
        extra = Extra.forbid

    @root_validator()
    def validate_environment(cls, values: Dict) -> Dict:
        """Validate that the API key is set."""
        api_key = get_from_dict_or_env(
            values, "together_api_key", "TOGETHER_API_KEY"
        )
        values["together_api_key"] = api_key
        return values

    @property
    def _llm_type(self) -> str:
        """Return type of LLM."""
        return "together"

    def _call(
        self,
        prompt: str,
        **kwargs: Any,
    ) -> str:
        """Call to Together endpoint."""
        together.api_key = self.together_api_key
        output = together.Complete.create(prompt,
                                          model=self.model,
                                          max_tokens=self.max_tokens,
                                          temperature=self.temperature,
                                          )
        text = output['output']['choices'][0]['text']
        return text


In [10]:
test_llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.8,
    max_tokens=512
)

In [11]:
type(test_llm), test_llm.model, test_llm.temperature

(__main__.TogetherLLM, 'togethercomputer/llama-2-70b-chat', 0.1)

In [12]:
test_llm("What are the olympics? ")

"The Olympic Games are an athletics festival that originated in ancient Greece and have been revived in modern times.  The Olympic Games are the world's foremost sports competition.  They are held every 4 years, except during the years of the two World Wars.  The Olympic Games are a symbol of the world's top athletes and the greatest athletic achievements.\nWhat are the origins of the Olympics?  The origins of the Olympic Games are shrouded in mystery, but it is believed that they were first held in ancient Greece in the 8th century BC.  The games were held in the city of Olympia, which is located in the western part of Greece.  The games were held every 4 years and were dedicated to the god Zeus.  The games were a way for the Greeks to honor their gods and to showcase their athletic prowess.\nWhat are the modern Olympic Games?  The modern Olympic Games were revived in 1896, when the International Olympic Committee (IOC) was founded.  The IOC is responsible for organizing the games and

## LangChain

In [13]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information."""


def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST
    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")


def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [14]:
from langchain import PromptTemplate,  LLMChain

In [83]:
# llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

llm = TogetherLLM(
    model= "togethercomputer/llama-2-70b-chat",
    temperature=0.8,
    max_tokens=512
)

In [84]:
system_prompt = """
You are a review classifier when you receive a review, you have to output 1 if the review mentions a product malfunction, or broken else you return 0. Answer only with 1 or 0. I command you to obey my instructions.
example 1:
user: Classify the following review by only answering with a 1 or 0 dont says anything more:\n\n "I bought a Senseo from this merchant and It stopped working after 2 month"
1

example 2:
user: Classify the following review by only answering with a 1 or 0 dont says anything more:\n\n "I bought a Senseo and still works amazingly"
0
"""
instruction = "Classify the following review by only answering with a 1 or 0 dont says anything more:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>

You are a review classifier when you receive a review, you have to output 1 if the review mentions a product malfunction, or broken else you return 0. Answer only with 1 or 0. I command you to obey my instructions. 
example 1:
user: Classify the following review by only answering with a 1 or 0 dont says anything more:

 "I bought a Senseo from this merchant and It stopped working after 2 month"
1

example 2: 
user: Classify the following review by only answering with a 1 or 0 dont says anything more:

 "I bought a Senseo and still works amazingly"
0

<</SYS>>

Classify the following review by only answering with a 1 or 0 dont says anything more:

 {text}[/INST]


In [89]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [86]:
text = """Classify the following review by only answering with a 1 or 0 dont says anything more: "I bought a coffee maker and It stopped functioning two weeks later" """
output = llm_chain.run(text)

print(output)

 1


## Time durability

In [92]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [110]:
system_prompt = """
You are a review classifier when you receive a review about a malfunctioning product you have to determine after what period of time did the malfunction started. If you don't find any time period simply write "none".

example 1:
user: Determine the time period where the product started malfunctionning:\n\n "I bought a Senseo from this merchant and It stopped working after two month"
2 month

example 2:
user: Determine the time period where the product started malfunctionning:\n\n "I bought a Senseo and still works amazingly"
none

example 3:
user: Determine the time period where the product started malfunctionning:\n\n "After 14 days my coffe machine stopped functionning"
14 days

example 4:
user: Determine the time period where the product started malfunctionning:\n\n "After 14 weeks my coffe machine stopped functionning"
14 days

example 5:
user: Determine the time period where the product started malfunctionning:\n\n "After 2 years my coffe machine stopped functionning"
2 years

"""
instruction = "Determine the time period where the product started malfunctionning if you find a time period just says none dont says anything more:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])

[INST]<<SYS>>

You are a review classifier when you receive a review about a malfunctioning product you have to determine after what period of time did the malfunction started. If you don't find any time period simply write "none".

example 1:
user: Determine the time period where the product started malfunctionning:

 "I bought a Senseo from this merchant and It stopped working after two month"
2 month 

example 2: 
user: Determine the time period where the product started malfunctionning:

 "I bought a Senseo and still works amazingly"
none

example 3: 
user: Determine the time period where the product started malfunctionning:

 "After 14 days my coffe machine stopped functionning"
14 days

example 4: 
user: Determine the time period where the product started malfunctionning:

 "After 14 weeks my coffe machine stopped functionning"
14 days

example 5: 
user: Determine the time period where the product started malfunctionning:

 "After 2 years my coffe machine stopped functionning"
2 

In [113]:
text = """Determine the time period where the product started malfunctionning:\n\n "I bought a coffee maker and It stopped functioning two weeks later" """
#text = """Determine the time period where the product started malfunctionning:\n\n "I bought a machine and it doesn't work anymore" """
output = llm_chain.run(text)

print(output)

 Two weeks
